# Using eval_metric in XGBoost 

This notebook shows how to use XGBoost's Evaluation Metric (`eval_metric`) with H2O. 



### What is eval_metric?

Original (non-H2O) XGBoost library allows users to define one or more evaluation metrics that will be calculated on both training and validations datasets after each iteration. If no evaluation metric is defined by the user XGBoost will assign it based on the choice of the objective function. For example in the case of binary classification, XGBoost will use logloss to report performance on training and validation data. Logloss measures the extent to which predicted probabilities diverge from the actual class labels. You might want to consider evaluating the performance using a different metric, e.g. for imbalanced problems, where it is more important to correctly predict the positive minority class, you might want to use the Area under the PR curve and specify `eval_metric="aucpr"`. Evaluation metric can easily be used for early stopping.

### Is eval_metric needed with H2O XGBoost?

H2O's approach to calculating metrics is different. By default, H2O will calculate all appropritate metrics for the given problem. For example for binary classification, H2O will report logloss, AUC, AUCPR and also additional metrics. Users thus typically don't need to worry about selecting the appropriate metric before model training. When early stopping is used, users will need to chose from one built-in early stopping metrics. For consistency between different model types and/or algorithm implementations, these are always calculated by H2O itself and, in XGBoost's specific case independect of XGBoost's eval_metric implementation.

### When should eval_metric be considered?

While you typically don't need to specify your custom `eval_metric`, there are cases doing so would be beneficial.

Case 1: H2O doesn't provide a suitable built-in metric. Example: If you want to calculate classification error for a different threshold than the one automatically determined by H2O, you can do so only by specifying `eval_metric="error@<your threshold>"` because H2O currently doesn't have this capability.

Case 2: Frequent scoring. By default H2O uses timing-triggered scoring, it is trying to make sure that majority of the time is used on model training as opposed to just model scoring. You can override this behavior and specify manually at what iterations you want model to be scored (eg. each iteration, every 5th iteration, ...). Because H2O calculates all possible metrics as opposed just few in native XGBoost, and needs to extract XGBoost model from the native memory, it can have a significant overhead when user desires to score very frequently. This can slow down the model building.

### Example

We will create a synthetic classification dataset, show XGBoost model training with default parameters and compare it to the output of model training with eval_metric used for early stopping.

In [1]:
import h2o
h2o.init(strict_version_check=False)

versionFromGradle='3.39.0',projectVersion='3.39.0.99999',branch='michalk_eval-metric-ntb',lastCommitHash='256ed83c89220493cca0574ebd517eb09e3611fd',gitDescribe='jenkins-master-5998-dirty',compiledOn='2022-10-28 14:15:26',compiledBy='kurkami'
Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,5 hours 14 mins
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.39.0.99999
H2O_cluster_version_age:,1 day
H2O_cluster_name:,kurkami
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.051 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [2]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=100000, 
                           n_informative=5,
                           n_classes=2,
                           random_state=42)
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.25)

In [3]:
from h2o import H2OFrame
train = H2OFrame(y_train, column_names=["y"]).cbind(H2OFrame(X_train))
valid = H2OFrame(y_validation, column_names=["y"]).cbind(H2OFrame(X_validation))
train["y"] = train["y"].asfactor()
valid["y"] = valid["y"].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [4]:
train

y,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20
1,-0.460911,0.587072,1.27653,1.65893,0.82732,0.900837,1.27825,-1.15342,-1.10419,0.231584,1.42246,0.632382,1.6532,0.0807537,0.63636,-0.0407703,-0.712485,-0.987504,0.529023,0.813985
0,-0.122517,0.926967,-3.27666,1.65648,1.26381,0.866601,-0.112373,4.88467,-2.56306,1.37596,1.66414,0.750439,-0.712589,0.011264,0.338901,-0.91639,-1.37998,-1.37817,-0.702391,-0.222837
1,1.07287,1.35581,0.504084,-1.9799,0.860703,-0.753541,1.17781,0.364174,2.49498,-0.309902,0.52471,-0.128492,0.133623,1.77568,-1.70823,0.263603,-1.69966,0.77875,-0.578858,1.53087
1,-0.497834,0.427476,-1.67148,-1.71013,1.11473,0.0850045,-2.81089,3.2892,0.676264,1.04835,1.46596,0.818714,-1.03108,-1.65528,0.301107,1.19965,0.120588,0.820176,-0.930966,-0.82086
1,0.923038,1.51161,1.66059,1.45418,-2.3419,-0.757573,0.308842,-3.57087,-0.939606,0.369506,0.990806,0.67289,1.53918,1.79014,0.726961,0.0339329,-0.542889,-1.01856,1.97659,1.50693
0,0.575923,2.20724,0.595904,-0.416675,0.21922,0.787082,-0.0753548,-0.413841,-0.280823,0.756811,-0.115625,-1.03631,0.617746,1.15633,-1.3063,1.83909,0.610228,0.62274,-0.157638,-1.19873
0,-0.931317,3.38117,-1.25809,0.323422,4.46615,0.366347,-0.178002,4.59046,-0.995683,0.248196,-3.7437,1.22383,-1.00021,-0.754449,0.573945,1.94618,-2.16485,-0.285859,-3.99979,-1.39355
0,0.956366,1.02616,-0.55144,-0.66649,2.47247,1.14188,1.10357,2.61941,1.2818,0.528047,-1.10656,-0.664716,-0.849829,1.49331,-0.183118,0.618475,-0.413351,0.410573,-2.14432,2.35245
0,-0.0523249,1.73885,-1.47901,0.254941,-1.37106,-0.0549625,-1.19023,0.848204,-0.424188,-0.0600663,-0.781962,1.1958,-0.456581,-0.740832,2.26734,2.01902,1.89101,0.348762,0.260157,0.133197
0,-0.818169,-0.746954,-1.28428,-0.0786964,0.708029,-0.252285,-0.19201,1.33074,0.229397,-0.599344,-0.141543,-0.721745,0.442618,0.953017,1.00111,-0.862009,0.790705,1.56841,0.101824,1.48008


##### Train XGBoost model with logloss as stopping metric

We are specifying 1000 trees to be built and providing stopping criteria to make the model stop reasonably early for the purpose of this example.

In [5]:
from h2o.estimators.xgboost import H2OXGBoostEstimator
model_def = H2OXGBoostEstimator(ntrees=1000, max_depth=6, score_each_iteration=True, 
                                stopping_rounds=3, stopping_tolerance=0.05, stopping_metric="logloss")
model_def.train(y="y", training_frame=train, validation_frame=valid)

xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_model_python_1666962310297_518


Model Summary: 
    number_of_trees
--  -----------------
    22

ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.03475774769265435
RMSE: 0.18643429859511995
LogLoss: 0.12901406890397824
Mean Per-Class Error: 0.04387881533186782
AUC: 0.9893340459497513
AUCPR: 0.988704085388505
Gini: 0.9786680918995025

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5101676438305829
       0      1      Error    Rate
-----  -----  -----  -------  ----------------
0      35155  2174   0.0582   (2174.0/37329.0)
1      1112   36559  0.0295   (1112.0/37671.0)
Total  36267  38733  0.0438   (3286.0/75000.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.510168     0.956992  214
max f2                       0.308281     0.971508  263
max f0point5                 0.753261     0.959208  147
max accuracy                 0.556171     0.9564    201
max precision                0.997456     1         0
max recall                   0.00160282   1         399
max specificity              0.997456     1         0
max absolute_mcc             0.556171     0.912928  201
max min_per_class_accuracy   0.623985     0.954807  184
max mean_per_class_accuracy  0.556171     0.95636   201
max tns                      0.997456     37329     0
max fns                      0.997456     37460     0
max fps                      0.00160282   37329     399
max tps                      0.00160282   37671     399
max tnr                      0.997456     1         0
max fnr                      0.997456     0.994399  0
max fpr                      0.00160282   1         399
max tpr                      0.00160282   1         399

Gains/Lift Table: Avg response rate: 50.23 %, avg score: 50.22 %
group    cumulative_data_fraction    lower_threshold    lift        cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ----------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0101733                   0.99611            1.99092     1.99092            1                0.996752    1                           0.996752            0.0202543       0.0202543                  99.0921   99.0921            0.0202543
2        0.0200267                   0.995715           1.99092     1.99092            1                0.995889    1                           0.996327            0.0196172       0.0398715                  99.0921   99.0921            0.0398715
3        0.03068                     0.995396           1.99092     1.99092            1                0.995549    1                           0.996057            0.0212099       0.0610815                  99.0921   99.0921            0.0610815
4        0.0401067                   0.995194           1.98811     1.99026            0.998586         0.995271    0.999668                    0.995872            0.0187412       0.0798227                  98.8105   99.026             0.0797959
5        0.0517733                   0.99493            1.98637     1.98938            0.997714         0.995008    0.999227                    0.995678            0.0231743       0.102997                   98.6371   98.9383            0.102917
6        0.100013                    0.99331            1.98817     1.9888             0.998618         0.994166    0.998933                    0.994948            0.0959093       0.198906                   98.817    98.8798            0.198692
7        0.15008                     0.99049

Scoring history will show us some of the metrics that were calculated on training and validation datasets. In our case, `validation_logloss` was used as the metric for early stopping.

In [6]:
model_def.scoring_history()

,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_pr_auc,validation_lift,validation_classification_error
0,,2022-10-28 14:19:27,0.031 sec,0.0,0.500000,0.693147,0.500000,0.502280,1.000000,0.497720,0.500000,0.693147,0.500000,0.492640,1.000000,0.50736
1,,2022-10-28 14:19:27,0.150 sec,1.0,0.399601,0.508456,0.959960,0.952882,1.969944,0.085080,0.400294,0.509577,0.958475,0.950121,2.007326,0.08816
2,,2022-10-28 14:19:27,0.213 sec,2.0,0.337078,0.403205,0.969336,0.963843,1.974978,0.078693,0.337873,0.404315,0.968964,0.962745,2.018078,0.08072
3,,2022-10-28 14:19:27,0.272 sec,3.0,0.295714,0.333565,0.973576,0.969297,1.982358,0.071240,0.297280,0.335569,0.972759,0.967269,2.016481,0.07328
4,,2022-10-28 14:19:27,0.337 sec,4.0,0.269057,0.286705,0.976939,0.973666,1.982440,0.068773,0.271193,0.289430,0.975368,0.970918,2.016613,0.07028
5,,2022-10-28 14:19:28,0.461 sec,5.0,0.252977,0.254899,0.978235,0.975398,1.983001,0.069360,0.255044,0.257526,0.976955,0.972420,2.020453,0.06956
6,,2022-10-28 14:19:28,0.553 sec,6.0,0.243055,0.232886,0.979254,0.976733,1.982694,0.068640,0.245257,0.235729,0.977808,0.973754,2.019880,0.06872
7,,2022-10-28 14:19:28,0.646 sec,7.0,0.236670,0.217304,0.980100,0.977952,1.984012,0.065533,0.238892,0.220206,0.978794,0.975109,2.018908,0.06696
8,,2022-10-28 14:19:28,0.742 sec,8.0,0.227428,0.200464,0.981689,0.979585,1.988342,0.061720,0.230286,0.204054,0.980333,0.977271,2.021951,0.06516
9,,2022-10-28 14:19:28,0.831 sec,9.0,0.223505,0.190888,0.982344,0.980522,1.988395,0.060800,0.226757,0.195105,0.980813,0.977909,2.022277,0.06432


##### Train XGBoost model and use eval_metric="logloss" for early stopping

We will use the same parameters as in the first case and add `eval_metric="logloss"`. To use actually use the value of the `eval_metric` for early stopping, we also need to specify `stopping_metric="custom"`.

In [7]:
model_eval = H2OXGBoostEstimator(ntrees=1000, max_depth=6, score_each_iteration=True, 
                                 eval_metric="logloss",
                                 stopping_rounds=3, stopping_tolerance=0.05, stopping_metric="custom")
model_eval.train(y="y", training_frame=train, validation_frame=valid)

xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_model_python_1666962310297_567


Model Summary: 
    number_of_trees
--  -----------------
    22

ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.03475774769265435
RMSE: 0.18643429859511995
LogLoss: 0.12901406890397824
Mean Per-Class Error: 0.04387881533186782
AUC: 0.9893340459497513
AUCPR: 0.988704085388505
Gini: 0.9786680918995025

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5101676438305829
       0      1      Error    Rate
-----  -----  -----  -------  ----------------
0      35155  2174   0.0582   (2174.0/37329.0)
1      1112   36559  0.0295   (1112.0/37671.0)
Total  36267  38733  0.0438   (3286.0/75000.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.510168     0.956992  214
max f2                       0.308281     0.971508  263
max f0point5                 0.753261     0.959208  147
max accuracy                 0.556171     0.9564    201
max precision                0.997456     1         0
max recall                   0.00160282   1         399
max specificity              0.997456     1         0
max absolute_mcc             0.556171     0.912928  201
max min_per_class_accuracy   0.623985     0.954807  184
max mean_per_class_accuracy  0.556171     0.95636   201
max tns                      0.997456     37329     0
max fns                      0.997456     37460     0
max fps                      0.00160282   37329     399
max tps                      0.00160282   37671     399
max tnr                      0.997456     1         0
max fnr                      0.997456     0.994399  0
max fpr                      0.00160282   1         399
max tpr                      0.00160282   1         399

Gains/Lift Table: Avg response rate: 50.23 %, avg score: 50.22 %
group    cumulative_data_fraction    lower_threshold    lift        cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ----------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0101733                   0.99611            1.99092     1.99092            1                0.996752    1                           0.996752            0.0202543       0.0202543                  99.0921   99.0921            0.0202543
2        0.0200267                   0.995715           1.99092     1.99092            1                0.995889    1                           0.996327            0.0196172       0.0398715                  99.0921   99.0921            0.0398715
3        0.03068                     0.995396           1.99092     1.99092            1                0.995549    1                           0.996057            0.0212099       0.0610815                  99.0921   99.0921            0.0610815
4        0.0401067                   0.995194           1.98811     1.99026            0.998586         0.995271    0.999668                    0.995872            0.0187412       0.0798227                  98.8105   99.026             0.0797959
5        0.0517733                   0.99493            1.98637     1.98938            0.997714         0.995008    0.999227                    0.995678            0.0231743       0.102997                   98.6371   98.9383            0.102917
6        0.100013                    0.99331            1.98817     1.9888             0.998618         0.994166    0.998933                    0.994948            0.0959093       0.198906                   98.817    98.8798            0.198692
7        0.15008                     0.99049

The scoring history for model with `eval_metric="logloss"`  will look similar to the scoring history of the first model. This is expected - we didn't actually changed the training behavior, we only changed the source of the values that trigger early stopping. In this case, we are stopping on values of `validation_custom`. This value correspond to the value calculated and returned by XGBoost. It should be close to H2O's own `validation_logloss` value, there can be only a small difference caused by a different precision in XGBoost and H2O (the values should be within absolute tolerance of 1e-5). This is, however, something to keep in mind. There can be edge cases where H2O metric will differ slightly from the conceptually same XGBoost metric and this might cause the models to stop at a different iteration.

The scoring history will also have value of `eval_metric` for the training frame - see column `training_custom`.

In [8]:
model_eval.scoring_history()

,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error,training_custom,validation_rmse,validation_logloss,validation_auc,validation_pr_auc,validation_lift,validation_classification_error,validation_custom
0,,2022-10-28 14:19:30,0.025 sec,0.0,0.500000,0.693147,0.500000,0.502280,1.000000,0.497720,0.693147,0.500000,0.693147,0.500000,0.492640,1.000000,0.50736,0.693147
1,,2022-10-28 14:19:30,0.139 sec,1.0,0.399601,0.508456,0.959960,0.952882,1.969944,0.085080,0.508456,0.400294,0.509577,0.958475,0.950121,2.007326,0.08816,0.509577
2,,2022-10-28 14:19:30,0.179 sec,2.0,0.337078,0.403205,0.969336,0.963843,1.974978,0.078693,0.403205,0.337873,0.404315,0.968964,0.962745,2.018078,0.08072,0.404315
3,,2022-10-28 14:19:30,0.228 sec,3.0,0.295714,0.333565,0.973576,0.969297,1.982358,0.071240,0.333565,0.297280,0.335569,0.972759,0.967269,2.016481,0.07328,0.335569
4,,2022-10-28 14:19:30,0.280 sec,4.0,0.269057,0.286705,0.976939,0.973666,1.982440,0.068773,0.286705,0.271193,0.289430,0.975368,0.970918,2.016613,0.07028,0.289430
5,,2022-10-28 14:19:30,0.356 sec,5.0,0.252977,0.254899,0.978235,0.975398,1.983001,0.069360,0.254899,0.255044,0.257526,0.976955,0.972420,2.020453,0.06956,0.257526
6,,2022-10-28 14:19:30,0.434 sec,6.0,0.243055,0.232886,0.979254,0.976733,1.982694,0.068640,0.232886,0.245257,0.235729,0.977808,0.973754,2.019880,0.06872,0.235729
7,,2022-10-28 14:19:30,0.511 sec,7.0,0.236670,0.217304,0.980100,0.977952,1.984012,0.065533,0.217304,0.238892,0.220206,0.978794,0.975109,2.018908,0.06696,0.220206
8,,2022-10-28 14:19:30,0.587 sec,8.0,0.227428,0.200464,0.981689,0.979585,1.988342,0.061720,0.200464,0.230286,0.204054,0.980333,0.977271,2.021951,0.06516,0.204054
9,,2022-10-28 14:19:31,0.676 sec,9.0,0.223505,0.190888,0.982344,0.980522,1.988395,0.060800,0.190888,0.226757,0.195105,0.980813,0.977909,2.022277,0.06432,0.195105


##### Train XGBoost model, use eval_metric="logloss" for early stopping and disable H2O metrics to speed-up model training

In this example we will keep the same parameters as in the previous case and add flag `score_eval_metric_only=True`. This flag will instruct H2O to disable its own scoring and solely rely on `eval_metric` for early stopping and recording the scoring history.

In [9]:
model_eval_only = H2OXGBoostEstimator(ntrees=1000, max_depth=6, score_each_iteration=True, 
                                      eval_metric="logloss",
                                      stopping_rounds=3, stopping_tolerance=0.05, stopping_metric="custom",
                                      score_eval_metric_only=True)
model_eval_only.train(y="y", training_frame=train, validation_frame=valid)

xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_model_python_1666962310297_616


Model Summary: 
    number_of_trees
--  -----------------
    22

ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.03475774769265435
RMSE: 0.18643429859511995
LogLoss: 0.12901406890397824
Mean Per-Class Error: 0.04387881533186782
AUC: 0.9893340459497513
AUCPR: 0.988704085388505
Gini: 0.9786680918995025

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5101676438305829
       0      1      Error    Rate
-----  -----  -----  -------  ----------------
0      35155  2174   0.0582   (2174.0/37329.0)
1      1112   36559  0.0295   (1112.0/37671.0)
Total  36267  38733  0.0438   (3286.0/75000.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.510168     0.956992  214
max f2                       0.308281     0.971508  263
max f0point5                 0.753261     0.959208  147
max accuracy                 0.556171     0.9564    201
max precision                0.997456     1         0
max recall                   0.00160282   1         399
max specificity              0.997456     1         0
max absolute_mcc             0.556171     0.912928  201
max min_per_class_accuracy   0.623985     0.954807  184
max mean_per_class_accuracy  0.556171     0.95636   201
max tns                      0.997456     37329     0
max fns                      0.997456     37460     0
max fps                      0.00160282   37329     399
max tps                      0.00160282   37671     399
max tnr                      0.997456     1         0
max fnr                      0.997456     0.994399  0
max fpr                      0.00160282   1         399
max tpr                      0.00160282   1         399

Gains/Lift Table: Avg response rate: 50.23 %, avg score: 50.22 %
group    cumulative_data_fraction    lower_threshold    lift        cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ----------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0101733                   0.99611            1.99092     1.99092            1                0.996752    1                           0.996752            0.0202543       0.0202543                  99.0921   99.0921            0.0202543
2        0.0200267                   0.995715           1.99092     1.99092            1                0.995889    1                           0.996327            0.0196172       0.0398715                  99.0921   99.0921            0.0398715
3        0.03068                     0.995396           1.99092     1.99092            1                0.995549    1                           0.996057            0.0212099       0.0610815                  99.0921   99.0921            0.0610815
4        0.0401067                   0.995194           1.98811     1.99026            0.998586         0.995271    0.999668                    0.995872            0.0187412       0.0798227                  98.8105   99.026             0.0797959
5        0.0517733                   0.99493            1.98637     1.98938            0.997714         0.995008    0.999227                    0.995678            0.0231743       0.102997                   98.6371   98.9383            0.102917
6        0.100013                    0.99331            1.98817     1.9888             0.998618         0.994166    0.998933                    0.994948            0.0959093       0.198906                   98.817    98.8798            0.198692
7        0.15008                     0.99049

Scoring history will show undefined values for H2O metrics for all scoring iteration except for the final one. Values of columns `training_custom` and `validation_custom` will be the only ones populated for all of the iterations.

In the final iteration H2O performs full scoring, that is why we see all values defined in the last row of the scoring history.

In [10]:
model_eval_only.scoring_history()

,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error,training_custom,validation_rmse,validation_logloss,validation_auc,validation_pr_auc,validation_lift,validation_classification_error,validation_custom
0,,2022-10-28 14:19:33,0.030 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.693147,NaN,NaN,NaN,NaN,NaN,NaN,0.693147
1,,2022-10-28 14:19:33,0.118 sec,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.508456,NaN,NaN,NaN,NaN,NaN,NaN,0.509577
2,,2022-10-28 14:19:33,0.127 sec,2.0,NaN,NaN,NaN,NaN,NaN,NaN,0.403205,NaN,NaN,NaN,NaN,NaN,NaN,0.404315
3,,2022-10-28 14:19:33,0.135 sec,3.0,NaN,NaN,NaN,NaN,NaN,NaN,0.333565,NaN,NaN,NaN,NaN,NaN,NaN,0.335569
4,,2022-10-28 14:19:33,0.144 sec,4.0,NaN,NaN,NaN,NaN,NaN,NaN,0.286705,NaN,NaN,NaN,NaN,NaN,NaN,0.289430
5,,2022-10-28 14:19:33,0.151 sec,5.0,NaN,NaN,NaN,NaN,NaN,NaN,0.254899,NaN,NaN,NaN,NaN,NaN,NaN,0.257526
6,,2022-10-28 14:19:33,0.161 sec,6.0,NaN,NaN,NaN,NaN,NaN,NaN,0.232886,NaN,NaN,NaN,NaN,NaN,NaN,0.235729
7,,2022-10-28 14:19:33,0.169 sec,7.0,NaN,NaN,NaN,NaN,NaN,NaN,0.217304,NaN,NaN,NaN,NaN,NaN,NaN,0.220206
8,,2022-10-28 14:19:33,0.177 sec,8.0,NaN,NaN,NaN,NaN,NaN,NaN,0.200464,NaN,NaN,NaN,NaN,NaN,NaN,0.204054
9,,2022-10-28 14:19:33,0.185 sec,9.0,NaN,NaN,NaN,NaN,NaN,NaN,0.190888,NaN,NaN,NaN,NaN,NaN,NaN,0.195105


Models `model_eval` and `model_eval_only` are guaranteed to be identical in behavior (same trees, same thresholds,..). The only technical difference between them is that the first one doesn't have full scoring history.

We can also see that flag `score_eval_metric_only=True` saved us some training time. Model `model_eval_only` was built faster:

In [11]:
def total_duration(scoring_history):
    return(sum(map(lambda x: float(x.strip().split(' ')[0]), scoring_history["duration"].tolist())))


print("Duration (s) with H2O scoring: %s" % total_duration(model_eval.scoring_history()))
print("Duration (s) with only eval_metric scored: %s" % total_duration(model_eval_only.scoring_history()))


Duration (s) with H2O scoring: 21.281
Duration (s) with only eval_metric scored: 4.67


In [12]:
model_eval.scoring_history()["duration"]

0      0.025 sec
1      0.139 sec
2      0.179 sec
3      0.228 sec
4      0.280 sec
5      0.356 sec
6      0.434 sec
7      0.511 sec
8      0.587 sec
9      0.676 sec
10     0.773 sec
11     0.854 sec
12     0.943 sec
13     1.058 sec
14     1.160 sec
15     1.246 sec
16     1.345 sec
17     1.462 sec
18     1.566 sec
19     1.684 sec
20     1.802 sec
21     1.916 sec
22     2.057 sec
Name: duration, dtype: object

In [13]:
model_eval_only.scoring_history()["duration"]

0      0.030 sec
1      0.118 sec
2      0.127 sec
3      0.135 sec
4      0.144 sec
5      0.151 sec
6      0.161 sec
7      0.169 sec
8      0.177 sec
9      0.185 sec
10     0.194 sec
11     0.202 sec
12     0.213 sec
13     0.224 sec
14     0.234 sec
15     0.243 sec
16     0.251 sec
17     0.261 sec
18     0.270 sec
19     0.279 sec
20     0.291 sec
21     0.301 sec
22     0.310 sec
Name: duration, dtype: object